<a href="https://colab.research.google.com/github/amalmon/RETAIL-SALES-PREDICTION/blob/main/AMAL_Rossmann_Sales_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [78]:
import pandas as pd
import numpy as np
from pandas import datetime as dt
from pandas import Series,DataFrame
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split
# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
rossmann_store_data= pd.read_csv('/content/drive/MyDrive/Almabetter/capstone projects/Capstone project 2 ML regression /Rossmann Stores Data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [80]:
store_data=pd.read_csv('/content/drive/MyDrive/Almabetter/capstone projects/Capstone project 2 ML regression /store.csv')

In [81]:
rossmann_df=rossmann_store_data.copy()

In [82]:
store_df=store_data.copy()

In [19]:
rossmann_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [20]:
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,0.0,0.0,0
1112,1113,a,c,9260.0,9.0,2013.0,0,0.0,0.0,0
1113,1114,a,c,870.0,9.0,2013.0,0,0.0,0.0,0


In [ ]:
rossmann_df.info()

In [ ]:
store_df.info()

In [ ]:
rossmann_df.describe(include='all')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,5,NaN
top,NaN,NaN,2015-07-31,NaN,NaN,NaN,NaN,0,NaN
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,855087,NaN
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00


In [ ]:
store_df.describe(include='all')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115.00000,1115,1115,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000,571
unique,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,a,a,NaN,NaN,NaN,NaN,NaN,NaN,"Jan,Apr,Jul,Oct"
freq,NaN,602,593,NaN,NaN,NaN,NaN,NaN,NaN,335
mean,558.00000,NaN,NaN,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573,NaN
std,322.01708,NaN,NaN,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935,NaN
min,1.00000,NaN,NaN,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000,NaN
25%,279.50000,NaN,NaN,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000,NaN
50%,558.00000,NaN,NaN,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000,NaN
75%,836.50000,NaN,NaN,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000,NaN


**for duplicate row**

In [ ]:
len(rossmann_df[rossmann_df.duplicated()])

0

In [ ]:
len(store_df[store_df.duplicated()])

0

In [ ]:
rossmann_df.DayOfWeek.value_counts()

In [ ]:
rossmann_df.Open.value_counts()

**For null values**

In [ ]:
rossmann_df.isnull().any()

In [ ]:
store_df.isnull().any()

In [11]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [ ]:
store_df.loc[store_df['CompetitionOpenSinceMonth'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
11,12,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
12,13,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
18,19,a,c,3240.0,NaN,NaN,1,22.0,2011.0,"Mar,Jun,Sept,Dec"
21,22,a,a,1040.0,NaN,NaN,1,22.0,2012.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
1095,1096,a,c,1130.0,NaN,NaN,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
1099,1100,a,a,540.0,NaN,NaN,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


**. Here we can understand that null values of competition since month & year is due to missing data why because they have fare values under competition distance.**

**. It doesn't mean that there is no competition**

In [ ]:
store_df.loc[store_df['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


**. Here we can assume that these 3 stores have no competitors**

So, we can replace NaN values of copetition distance column with maximum value of competition distance available in data
And also we can replace NaN values of column competition since month and year with mode values of the respective columns

In [ ]:
store_df['CompetitionDistance'].max()

75860.0

In [ ]:
store_df['CompetitionDistance'].min()

20.0

In [ ]:
store_df['CompetitionDistance'].mean()

5594.466367713005

In [ ]:
store_df['CompetitionDistance'].median()

2330.0

*here may be no competetors at all for the shopes with null values of competition distance bt based on that when we applying maximum values there is a increase in outliers in column value. so, we choose mean.
and also it will not make significant changes in dataset

In [83]:
# null vales of column CompetitionDistance replaced by mean value
store_df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].mean(), inplace = True)

In [ ]:
#checking

In [ ]:
store_df.loc[290]


In [ ]:
# understanding values of CompetitionOpenSinceMonth

store_df.CompetitionOpenSinceMonth.value_counts()

In [ ]:
#taking mode value of CompetitionOpenSinceMonth

store_df.CompetitionOpenSinceMonth.mode()[0]

In [84]:
#replacing null values of CompetitionOpenSinceMonth column with its mode value

store_df['CompetitionOpenSinceMonth'].fillna(store_df.CompetitionOpenSinceMonth.mode()[0], inplace = True)

In [ ]:
#checking the changes happen to dataset

store_df.info()

In [85]:
#replacing null values of CompetitionOpenSinceYear with mode value

store_df['CompetitionOpenSinceYear'].fillna(store_df.CompetitionOpenSinceYear.mode()[0], inplace = True)

In [ ]:
#checking the changes happen

store_df.info()

In [ ]:
#understanding promo2 column values and its counts

store_df.Promo2.value_counts()

1    571
0    544
Name: Promo2, dtype: int64

In [ ]:
#for mode value of Promo2SinceWeek
store_df.Promo2SinceWeek.mode()

In [ ]:
#understanding Promo2SinceWeek column values and its counts
store_df.Promo2SinceWeek.value_counts()

In [ ]:
store_df.loc[store_df['Promo2SinceWeek'].isnull()]

**.Here we can conform that the stores which do not participate in promo2 they do not have 
any values in promo2 since yr & month and also promo2 interval**

**. so we can replace the NaN values of promo2 since [yr& month] and promo 2 interval with value 'zero'**

In [ ]:
''' imparting zero value to the null values
in columns promo2 since [yr& month] and promo 2 interval in the rows which contain zero value in promo2 column'''

In [ ]:
store_df.shape[0]

In [15]:
#putting zero in promosince week where promo 2 is zero

for i in range(0,store_df.shape[0]):
  if store_df.iloc[i,6]==0:
    store_df.iloc[i,7]=0

* null values of  promo2 since [yr& month] and promo 2 interval can be also directly make it as 'zero' 

In [86]:
#replacing null values of column Promo2SinceYear & PromoInterval with zero
store_df['Promo2SinceWeek'].fillna(0,inplace=True)
store_df['Promo2SinceYear'].fillna(0,inplace=True)
store_df['PromoInterval'].fillna(0,inplace=True)


In [ ]:
store_df.info()

In [ ]:
final_df

In [ ]:
final_df.info()

In [ ]:
final_df.columns

In [ ]:
final_df.PromoInterval.value_counts()

In [ ]:
'''#creating bar plot of promo intervel with averg. sales
ax=final_df.groupby(['PromoInterval'])['Sales'].sum().plot.bar(figsize=(15,5),fontsize=15)

#giving tittle
ax.set_title('average sales versus promotion intervals',fontsize=15)'''




In [ ]:
final_df=pd.get_dummies(final_df,columns=['StoreType','Assortment','PromoInterval'])

In [ ]:
final_df.PromoInterval.value_counts()

In [ ]:
final_df.Assortment.value_counts()

In [ ]:
final_df

In [ ]:
final_df.drop('PromoInterval_0',axis=1,inplace=True)

In [87]:
rossmann_df["Date"]=pd.to_datetime(rossmann_df["Date"])
#final_df["Year"]=final_df["Date"].dt.year
#final_df["Month"]=final_df["Date"].dt.month
#final_df["Day"]=final_df["Date"].dt.day
#final_df["Week"]=final_df["Date"].dt.week%4

In [88]:
rossmann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Store          1017209 non-null  int64         
 1   DayOfWeek      1017209 non-null  int64         
 2   Date           1017209 non-null  datetime64[ns]
 3   Sales          1017209 non-null  int64         
 4   Customers      1017209 non-null  int64         
 5   Open           1017209 non-null  int64         
 6   Promo          1017209 non-null  int64         
 7   StateHoliday   1017209 non-null  object        
 8   SchoolHoliday  1017209 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 69.8+ MB


In [ ]:
store_df.info()

In [95]:
store_df['CompetitionOpenSinceYear']=store_df['CompetitionOpenSinceYear'].astype(int)
store_df['CompetitionOpenSinceMonth']=store_df['CompetitionOpenSinceMonth'].astype(int)

In [ ]:
store_df.CompetitionOpenSinceMonth.value_counts()

In [96]:
#making new column by combining competition since month and year (date assumed to be 15)
store_df['competition since'] =pd.to_datetime(store_df['CompetitionOpenSinceYear'].astype(str)+'-'+store_df['CompetitionOpenSinceMonth'].astype(str))

In [100]:
del store_df['CompetitionOpenSinceMonth']

In [101]:
del store_df['CompetitionOpenSinceYear']

In [ ]:
store_df

In [103]:
# merging 2 datasets on store number

final_df = rossmann_df.merge(store_df,how='inner',on = ['Store'])

In [104]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition since
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01


In [ ]:
final_df.info()

In [108]:
final_df['competition before'] =(final_df['Date']-final_df['competition since'])/ np.timedelta64()

In [113]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition since,competition before
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,inf
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,inf
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,inf
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,inf
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-inf
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-inf
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-inf
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-inf


In [ ]:
#taking difference of date and competition since date and making a single column with competition starts before how many days


Here we have promo since week and year.
we have to convert it into a date format 

Here week represents number of week in a calender year so, we have to convert it into month and day